# RNN Based molecule generation

Laurent Cetinsoy

In this hands-on we want to generate molecule formulas for denovo-drug discovery.

For that we need to use Generative models. Generative models are models which goes beyond classification or simple regression : they are able to generate data that look like previously seens dataset.

There exists a lot of models :

- Bayesian models like graphical models
- Recurrent models (for sequence generation like texte)
- Variational auto encoders
- Generative adversarial models
- Flow and diffusion models


In the hands-on we will start by  trainning a character based RNN to generate smile molecules


We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one.

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N"
y = C

and ask the RNN to learn to predict y given X

Like a standard language model !


## RNN Language model


A language model is a model which predict the next token of a sequence given the previous ones :

$ P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p})  $


This model can be learned with a Recurrent neural network

$ y = P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p}) = RNN_{\theta} (X_{t-1}, X_{t-2}, ..., X_{t-p})  $


In order to train such model you need a corpus of data.



There are two main ways to do that : Word level model or character level model

For character level models, an interesting resource is : http://karpathy.github.io/2015/05/21/rnn-effectiveness/



Explain briefly what is the difference between word based language model and character based language model

Word-based Language Models: These models treat whole words as the fundamental unit (token). The vocabulary consists of all unique words in the dataset, and the model predicts the next word in a sequence (e.g., predicting "discovery" after "drug").

Character-based Language Models: These models treat individual characters as the fundamental unit. The vocabulary is much smaller (consisting of letters, numbers, and symbols), and the model predicts the next character one by one (e.g., predicting "C" after "C(=O)").

## Loading the data

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [1]:
!wget -O 250k_smiles.csv https://raw.githubusercontent.com/joeymach/Leveraging-VAE-to-generate-molecules/master/250k_smiles.csv

--2025-12-07 22:25:27--  https://raw.githubusercontent.com/joeymach/Leveraging-VAE-to-generate-molecules/master/250k_smiles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22606589 (22M) [text/plain]
Saving to: ‘250k_smiles.csv’

250k_smiles.csv     100%[===================>]  21.56M  19.0MB/s    in 1.1s    

2025-12-07 22:25:28 (19.0 MB/s) - ‘250k_smiles.csv’ saved [22606589/22606589]



Import pandas and load the first 1000 lines

In [2]:
import pandas as pd

df = pd.read_csv('250k_smiles.csv', nrows=1000)

Display the first rows of the dataframe

In [3]:
df.head()

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


## Processing the data

We need to do the following things :

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [4]:
max_len = df['smiles'].apply(len).max()
print(f"The longest molecule has {max_len} characters.")

The longest molecule has 106 characters.



Code a function **unic_characters(string)** which return the unic characters in a string


In [5]:
def unic_characters(string):
    return sorted(list(set(string)))

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters

In [6]:
all_smiles = "".join(df['smiles'].values)

vocab = unic_characters(all_smiles)
print(f"Unique characters found: {vocab}")

Unique characters found: ['\n', '#', '(', ')', '+', '-', '/', '1', '2', '3', '4', '5', '6', '7', '=', '@', 'B', 'C', 'F', 'H', 'I', 'N', 'O', 'S', '[', '\\', ']', 'c', 'l', 'n', 'o', 'r', 's']


Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value.
Add a character to specify the end of the molecule (like "\n")


In [7]:
def map_char_to_int(unic_chars):
    char_to_int = {char: i for i, char in enumerate(unic_chars)}
    
    if "\n" not in char_to_int:
        char_to_int["\n"] = len(char_to_int)
        
    return char_to_int

char_to_int = map_char_to_int(vocab)

Code a function map_int_to_char(unic_chars) which returns the reverse mapping.

If you want you can merge both functions in a class

In [8]:
def map_int_to_char(char_to_int_map):
    int_to_char = {i: char for char, i in char_to_int_map.items()}
    return int_to_char

int_to_char = map_int_to_char(char_to_int)

For each smile molecule add the ending token to it

In [9]:
df['smiles'] = df['smiles'].apply(lambda x: x + "\n")

print(df['smiles'].head())

0          CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n\n
1     C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n\n
2    N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...
3    CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...
4    N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...
Name: smiles, dtype: object


## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ?

Since the instruction asks to reshape the array to (n_examples, seq_length, 1), n_features will be 1. Instead of One-Hot Encoding (which would make n_features = len(vocab)), this specific tutorial treats the character indices as a single normalized numerical feature.

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position.


It should create X by by getting all character between i and i + seq_length
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [10]:
def build_X_and_y(string, i_char, seq_length):
    x_seq = string[i_char : i_char + seq_length]
    y_char = string[i_char + seq_length]
    
    return x_seq, y_char

test_string = "CC(=O)NC1"
seq_len_test = 4
for i in [1, 2, 3]:
    x_test, y_test = build_X_and_y(test_string, i, seq_len_test)
    print(f"i={i}: X='{x_test}', y='{y_test}'")

i=1: X='C(=O', y=')'
i=2: X='(=O)', y='N'
i=3: X='=O)N', y='C'


By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train

In [11]:
import numpy as np

seq_length = 40 

X_train = []
y_train = []

for smile in df['smiles']:
    if len(smile) > seq_length:
        for i in range(len(smile) - seq_length):
            x_seq, y_char = build_X_and_y(smile, i, seq_length)
            
            x_seq_int = [char_to_int[c] for c in x_seq]
            y_char_int = char_to_int[y_char]
            
            X_train.append(x_seq_int)
            y_train.append(y_char_int)

print(f"Total training examples: {len(X_train)}")

Total training examples: 7528


Create numpy arrays from the lists

In [12]:
X_train = np.array(X_train)
y_train = np.array(y_train)

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [13]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train = X_train.reshape(X_train.shape[0], seq_length, 1)

Normalize X by dividing each values by the total number of unic characters

In [14]:
X_train = X_train / len(char_to_int)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (7528, 40, 1)
y_train shape: (7528,)


Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ?


Add a Dense layer on top with with the appropriate activation function and number of neurones


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()

# First LSTM layer with return_sequences=True to feed the next LSTM
model.add(LSTM(128, input_shape=(seq_length, 1), return_sequences=True))
model.add(Dropout(0.2))

# Second LSTM layer (return_sequences=False is default, which is what we want for the last recurrent layer)
model.add(LSTM(128))
model.add(Dropout(0.2))

# Dense output layer
# Units = len(char_to_int) because we are predicting probability for each possible character
model.add(Dense(len(char_to_int), activation='softmax'))

model.summary()

/Users/toutane/epita/scia/mlbio/mlbio-practicals/practical-3/.pixi/envs/default/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 40, 128)        │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 33)             │         4,257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,401 (790.63 KB)

 Trainable params: 202,401 (790.63 KB)

 Non-trainable params: 0 (0.00 B)

Compile the model with the appropriate loss function and the adam optimizer

In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits !

In [17]:
X_tiny = X_train[:10]
y_tiny = y_train[:10]

#history = model.fit(X_tiny, y_tiny, epochs=20, verbose=1)

If it does not overfit try to fix data prep and model architecture so it does

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


In [19]:
def make_prediction(seed_start):
    generated_smile = seed_start
    pattern = [char_to_int[c] for c in seed_start]
    
    for i in range(100):
        
        x_input = np.array(pattern[-seq_length:])
        x_input = x_input.reshape(1, len(x_input), 1)
        x_input = x_input / len(char_to_int)
        
        prediction = model.predict(x_input, verbose=0)
        
        index = np.argmax(prediction)
        result_char = int_to_char[index]
        
        generated_smile += result_char
        pattern.append(index)
        
        if result_char == "\n":
            break
            
    return generated_smile

generate a molecule of your overfitted model

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("model_weights.keras", monitor='loss', verbose=1, save_best_only=False, mode='min')

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256

it should take a long time so either follow the class or go take a nap

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=256, callbacks=[checkpoint])

Generate between 100 and 1000 molecules.

create a list where molecules have between 10 and 50 atoms

In [ ]:
generated_molecules = []

for i in range(100):
    start_index = np.random.randint(0, len(df['smiles']) - 1)
    seed_seq = df['smiles'][start_index][:seq_length]
    
    mol = make_prediction(seed_seq)
    
    if 10 <= len(mol) <= 50:
        generated_molecules.append(mol)

print(f"Generated {len(generated_molecules)} valid-length molecules.")
print(generated_molecules[:5])

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

In [ ]:
from rdkit import Chem
from rdkit.Chem import QED

valid_molecules = []
qed_scores = []

for smi in generated_molecules:
    smi = smi.strip()
    
    mol = Chem.MolFromSmiles(smi)
    
    if mol is not None:
        score = QED.qed(mol)
        valid_molecules.append(smi)
        qed_scores.append(score)

import pandas as pd
results = pd.DataFrame({'SMILES': valid_molecules, 'QED': qed_scores})
print(results.head())
print(f"Average QED: {results['QED'].mean()}")

Bonus 1 : Using rdkit, compute the quantitative estimation of drug-likeness (QED) of your generated molecules.

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better